In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import numpy as np

In [ ]:
# Model definition
number_raster_layers = 9
number_pixels_layer = 19

class CNNRegressor(nn.Module):
    def __init__(self):
        super(CNNRegressor, self).__init__()
        self.conv1 = nn.Conv2d(number_raster_layers, 16, kernel_size=3, stride=1, padding=1)
        self.selu1 = nn.SELU()
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.selu2 = nn.SELU()
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.selu3 = nn.SELU()
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.selu4 = nn.SELU()
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.selu5 = nn.SELU()
        self.conv6 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.selu6 = nn.SELU()
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(512 * number_pixels_layer * number_pixels_layer, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.selu1(x)
        x = self.conv2(x)
        x = self.selu2(x)
        x = self.conv3(x)
        x = self.selu3(x)
        x = self.conv4(x)
        x = self.selu4(x)
        x = self.conv5(x)
        x = self.selu5(x)
        x = self.conv6(x)
        x = self.selu6(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

model = CNNRegressor()

In [ ]:
# Load model
path_model = '../../Data/Calibrated_models/global_regressor_V0.pth'
model.load_state_dict(torch.load(path_model))

In [ ]:
# Cambia el modelo al modo de evaluación (si es necesario)
model.eval()

In [ ]:
# Load tensors
path_tensor_train = '../../Data/Calibrated_models/global_regressor_V0_tensor_y_train.pth'
y_train = torch.load(path_tensor_train)

path_tensor_test = '../../Data/Calibrated_models/global_regressor_V0_tensor_y_test.pth'
y_test = torch.load(path_tensor_test)

path_tensor_test = '../../Data/Calibrated_models/global_regressor_V0_test_tensor.pth'
test_tensor = torch.load(path_tensor_test)

path_tensor_training = '../../Data/Calibrated_models/global_regressor_V0_training_tensor.pth'
training_tensor = torch.load(path_tensor_training)

In [ ]:
# Model eval
y_pred = model(test_tensor.to(torch.float))
y_true = y_test.to(torch.float)

# MSE
mse = nn.MSELoss()
mse_loss = mse(y_pred, y_true)
print("MSE:", mse_loss.item())

# RMSE
rmse_loss = torch.sqrt(mse_loss)
print("RMSE:", rmse_loss.item())

# MAE
mae_loss = torch.mean(torch.abs(y_pred - y_true))
print("MAE:", mae_loss.item())

y_true_array = y_true.flatten().tolist()
y_pred_array = y_pred.flatten().tolist()

# R^2
r2 = r2_score(y_true_array, y_pred_array)
print("R^2:", r2)

# Pearson
corr, p_value_r = pearsonr(y_true_array, y_pred_array)
print("Pearson's r:", corr)
print("p-value (Pearson):", p_value_r)
print("Sanity check: r * r?", corr*corr)
print("Sanity check: r2 - r * r = 0?", r2 == corr*corr, r2 - corr*corr)

# Spearman
rho, p_value_rho = spearmanr(y_true_array, y_pred_array)
print("Spearman's rho:", rho)
print("p-value:", p_value_rho)

In [ ]:
# Add model predictions to training and test sets
# Load training and test info

df_train = pd.read_csv('../../Data/data_train_new.csv')
df_test = pd.read_csv('../../Data/data_test_new.csv')

rows_with_na = df_test.isna().any(axis=1)
indices_na = df_test.index[rows_with_na]
df_test = df_test.drop(index=indices_na)

In [ ]:
# Model results for test partition
y_pred = model(test_tensor.to(torch.float))
# Turn the tensor into a NumPy array
y_pred_numpy_array = y_pred.detach().numpy()

# create a pandas series from NumPy array
y_pred_pandas_serie = pd.Series(y_pred_numpy_array.squeeze(), name='global_regressor_V0')
df_test['global_regressor_V0'] = y_pred_pandas_serie

# Save info
df_test.to_csv('../../Data/data_test_global_regressor_V0.csv', index=False)

In [ ]:
# Model results for train partition
y_train_predicted = model(training_tensor.to(torch.float))
# Turn the tensor into a NumPy array
y_train_predicted_numpy_array = y_train_predicted.detach().numpy()

# Select elements in positions: 0, 4, 8, 12, ...
indices = np.arange(0, len(y_train_predicted_numpy_array), 4)
y_train_predicted_numpy_array_filtered = y_train_predicted_numpy_array[indices]

# create a pandas series from NumPy array
y_train_predicted_pandas_serie = pd.Series(y_train_predicted_numpy_array_filtered.squeeze(), name='global_regressor_V0')
df_train['global_regressor_V0'] = y_train_predicted_pandas_serie

# Save info
df_train.to_csv('../../Data/data_train_global_regressor_V0.csv', index=False)